<h1>Pokewar on the go</h1>

In [90]:
import pygame
from PIL import Image
import pyodbc
import os
import sys
import pygame_gui
import random
import pandas as pd

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-2T77F44\SQLEXPRESS;'
                      'Database=DB_PokeWar;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

SCREEN_HEIGHT, SCREEN_LENGTH = (600, 750)
STAB = pd.read_csv('scripts/STAB.csv', sep=',')
PATH = 'D:/Sequence/Compilers/Python/Projects/PokewarRework/'

PINK = (249, 102, 102)
CRYAN = (120, 149, 178)
GREEN = (54, 174, 124)
RED = (255, 30, 0)
BLUE = (33, 146, 255)

**userPackage** and **hoverPackage** include <br>
['id': id of the user] - class string <br>
['team': team name of user] - class char <br>
<br>
**currentPlayerPackage** include <br>
['id': id of the user] - class string <br>
['speed': speed stat of user] - class int <br>  
['team': team name of user] - class char <br>
<br>
**userUnpackage** include <br>
['pokemon': pokemon of the user] - class Pokemon <br>
['team': team of user] - class Team <br>
<br>
**posPack** include <br>
['pokemon id': id of pokemon of the user] - class string <br>
['pos': position of user] - class tuple <br>

<h2>Other principal</h2>

In [91]:
def Scale2xGif(object):
    return [pygame.transform.scale2x(frame) for frame in object]

def ScaleTimes(object, time):
    for _ in range(time):
        object = Scale2xGif(object)
    return object

def GetSTAB(eleAttack, eleDefense):
    for i in range(len(STAB['element'])):
        if STAB['element'][i]==eleAttack:
            return STAB[eleDefense][i]

def CheckingStab(user, target):
    stab = 1
    for userEle in user.Element:
        for targetEle in target.Element:
            stab *= GetSTAB(userEle, targetEle)
    return stab

def UnpackPackage(Package, A, B):
    userTeam = A if Package['team']=='A' else B
    user = getPokemonById(Package['id'], A, B, team=userTeam.team)
    return {'pokemon': user, 'team': userTeam}

def GetPercentage(value, portion):
    return portion*value/100

def GetChance(chance):
    return True if random.randint(1,100)<=chance else False

def getPokemonById(id, A, B, team='none', index=-1):
    if (team!='none'):
        for pokemon in team:
            if (pokemon.id == id):
                return pokemon
    if (team!='none' and index!=-1): 
        return team[index]
    for pokemonA, pokemonB in zip(A.team, B.team):
        if (pokemonA.id == id):
            return pokemonA
        if (pokemonB.id == id):
            return pokemonB

def getTeamById(id, A, B):
    return A if id=='A' else B

def isAlive(pokemon):
    return True if pokemon.hp>0 else False

def checkEffectExist(effect, effectList):
    for eff in effectList:
        if eff.name==effect.name:
            return True
    return False

<h2>Blit specific part</h2>

In [92]:
def split_animated_gif(gif_file_path):
    ret = []
    gif = Image.open(gif_file_path)
    for frame_index in range(gif.n_frames):
        gif.seek(frame_index)
        frame_rgba = gif.convert("RGBA")
        pygame_image = pygame.image.fromstring(
            frame_rgba.tobytes(), frame_rgba.size, frame_rgba.mode
        )
        ret.append(pygame_image)
    return {'list': ret, 'frame': 0}
    

def BlitGif(screen, object, pos, speed=1.0, isOneAction=False, scalePoint='midbottom'):
    if object['frame']>=len(object['list']):
        object['frame'] = len(object['list'])-1 if isOneAction else 0
    img = object['list'][int(object['frame'])]
    rect = img.get_rect(midbottom=pos) if scalePoint=='midbottom' else img.get_rect(center=pos)
    screen.blit(img, rect)
    object['frame']+=speed
    return object


def BlitBased(screen, pokemon, pos):
    rect = pokemon.Based.get_rect(midbottom=(pos[0], pos[1]))
    screen.blit(pokemon.Based, rect)


def BlitText(screen, text, pos, size=20,color=(255,255,255), FontFamily='sans'):
    font=pygame.font.SysFont(FontFamily, size)
    string = font.render(text, True, color)
    screen.blit(string, pos)


def BlitBox(screen, pos, size, color=(0,0,0), border=5, borderColor=(255,255,255)):
    boxX, boxY = pos
    boxWidth, boxHeight = size
    pygame.draw.rect(screen, borderColor, (boxX,boxY,boxWidth,boxHeight))
    pygame.draw.rect(screen, color, (boxX+border, boxY+border, boxWidth-(border*2), boxHeight-(border*2)))


def Blit_mutiline_text(surface, text, pos, limit, size=20, FontFamily='sans', color=(255,255,255)):
    font = pygame.font.SysFont(FontFamily, size)
    words = [word.split(' ') for word in text.splitlines()]
    space = font.size(' ')[0]
    max_width, max_height = limit
    x, y = pos
    for line in words:
        for word in line:
            word_surface = font.render(word, 0, color)
            word_width, word_height = word_surface.get_size()
            if x + word_width >= max_width:
                x = pos[0]
                y += word_height
            surface.blit(word_surface, (x, y))
            x += word_width + space
        x = pos[0]
        y += word_height


<h2>Animation class</h2>

In [93]:
# a animate box must contain [skill, pokemon, team]
class Animation:
    def __init__(self):
        self.animateBox = []
        self.isFinish = False

    def SetScreen(self, screen):
        self.screen = screen

    def Add_to_animateBox(self, animatePack):
        if animatePack['skill'].category == 'C': return
        else: animatePack['skill'].animationSprite['frame'] = 0
        self.animateBox.append(animatePack)

    def CheckTime(self):
        if self.isFinish:
            self.animateBox.clear()
    
    def BlitAnimation(self):
        self.isFinish = True
        for animation in self.animateBox:
            pos = animation['team'].getConstPos(animation['pokemon'])
            animation['skill'].animationSprite = BlitGif(self.screen, animation['skill'].animationSprite, (pos[0], pos[1]+20), isOneAction=True, speed=0.3)
            if animation['skill'].animationSprite['frame'] < len(animation['skill'].animationSprite['list'])-1: 
                self.isFinish = False
        self.CheckTime()

    def Add_animatebox_with_package(self, skill, package, A, B):
        Pack = UnpackPackage(package, A, B)
        self.Add_to_animateBox({'skill': skill,
                            'pokemon': Pack['pokemon'],
                            'team': Pack['team']})

    def Add_animatebox_with_unpackage(self, skill, unpackage, A, B):
        self.Add_to_animateBox({'skill': skill,
                            'pokemon': unpackage['pokemon'],
                            'team': unpackage['team']})
ANIMATION = Animation()

<h2>Message class</h2>

In [94]:
# a mess box must contain [message, pokemon, team, type=?]
class Message:
    def __init__(self):
        self.messBox = []
        self.duration = 200
        self.speed = 1
        self.startTime = 0
        self._circle_cache = {}

    def _circlepoints(self, r):
        r = int(round(r))
        if r in self._circle_cache:
            return self. _circle_cache[r]
        x, y, e = r, 0, 1 - r
        self._circle_cache[r] = points = []
        while x >= y:
            points.append((x, y))
            y += 1
            if e < 0:
                e += 2 * y - 1
            else:
                x -= 1
                e += 2 * (y - x) - 1
        points += [(y, x) for x, y in points if x > y]
        points += [(-x, y) for x, y in points if x]
        points += [(x, -y) for x, y in points if y]
        points.sort()
        return points

    def render(self, text, font, gfcolor=pygame.Color('dodgerblue'), ocolor=(255, 255, 255), opx=2):
        textsurface = font.render(text, True, gfcolor).convert_alpha()
        w = textsurface.get_width() + 2 * opx
        h = font.get_height()
        osurf = pygame.Surface((w, h + 2 * opx)).convert_alpha()
        osurf.fill((0, 0, 0, 0))
        surf = osurf.copy()
        osurf.blit(font.render(text, True, ocolor).convert_alpha(), (0, 0))
        for dx, dy in self._circlepoints(opx):
            surf.blit(osurf, (dx + opx, dy + opx))
        surf.blit(textsurface, (opx, opx))
        self._circle_cache.clear()
        return surf, textsurface.get_width()

    def SetScreen(self, screen):
        self.screen = screen

    def Add_to_messBox(self, messPack):
        messPack['random'] = random.randint(-15, 15)
        self.messBox.append(messPack)

    def CheckTime(self):
        self.startTime+=self.speed
        if (self.startTime>=self.duration):
            self.messBox.clear()
            self.startTime=0 
    
    def BlitMess(self):
        self.CheckTime()
        font = pygame.font.SysFont('sans', 15)
        for mess in self.messBox:
            string = mess['message']
            pos = mess['team'].getConstPos(mess['pokemon'])
            text, width = self.render(mess['message'], font, gfcolor=mess['color'])
            self.screen.blit(text, ((pos[0]+mess['random'])-width/2, (pos[1]+mess['random'])-(self.startTime)))

    def Add_messbox_with_package(self, text, package, A, B, color=PINK):
        messPack = UnpackPackage(package, A, B)
        self.Add_to_messBox({'message': text,
                            'pokemon': messPack['pokemon'],
                            'team': messPack['team'],
                            'color': color})

    def Add_messbox_with_unpackage(self, text, unpackage, A, B, color=PINK):
        self.Add_to_messBox({'message': text,
                            'pokemon': unpackage['pokemon'],
                            'team': unpackage['team'],
                            'color': color})
MESS = Message()

<h2>Effect class</h2>

In [95]:
class Effect:
    def __init__(self, name, rate, rangeEffect):
        self.name = name
        self.loadEffect()
        self.rate = rate
        self.rangeEffect=rangeEffect
        try:
            self.sprite = pygame.image.load(PATH+'source/Effects/'+self.name+'.png').convert()
            self.sprite = pygame.transform.scale(self.sprite, (20,20))
        except:
            self.sprite = 'none'
            
    def loadEffect(self):
        sql = "SELECT * FROM Effect WHERE effectName=\'"+self.name+"\'"
        cursor.execute(sql)
        for cur in cursor:
            name, self.describe, self.typeEffect = tuple(cur)
    

<h2>Skill class</h2>

In [96]:
class Skill:
    def __init__(self, skillId) -> None:
        self.id = skillId
        self.loadSkillInfor()
        self.loadSkillStats()
        self.loadEffectSkill()
        self.count = 0
        self.isAvailable = True
        if self.category!='C': 
            self.animationSprite = split_animated_gif(PATH+'source/vfx/SkillAnimation/' + self.name + '.gif')
            self.animationSprite['list'] = Scale2xGif(self.animationSprite['list'])
        # self.skillSprite = pygame.image.load(path).convert_alpha()

    def loadSkillInfor(self):
        sql = "SELECT * FROM Skill WHERE skillId=\'"+self.id+"\'"
        cursor.execute(sql)
        for cur in cursor: 
            id, self.name, self.kind, self.eleId, self.explain, self.category = tuple(cur)

    def loadSkillStats(self):
        sql = "SELECT * FROM SkillInfor WHERE skillId=\'"+self.id+"\'"
        cursor.execute(sql)
        for cur in cursor:
            id, self.damage, self.accuracy, self.countdown, self.range = tuple(cur)

    def loadEffectSkill(self):
        sql = "SELECT * FROM SkillEffect WHERE skillId=\'"+self.id+"\'"
        cursor.execute(sql)
        self.EffectList = []
        for cur in cursor:
            self.EffectList.append(tuple(cur))
        self.EffectList = [Effect(cur[1], cur[2], cur[3]) for cur in self.EffectList]

    def setCountDownSkill_useSkill(self):
        self.count = self.countdown
        self.isAvailable = False

    def setCountDownSkill_getTurn(self):
        self.count-=1
        self.isAvailable = True if self.count <= 0 else False

<h2>Pokemon class</h2>

In [97]:
class Pokemon:
    def __init__(self, id):
        self.FrontSprite = split_animated_gif('source/Pokemon/'+id+'/Front.gif')
        self.BackSprite = split_animated_gif('source/Pokemon/'+id+'/Back.gif')
        self.StatusSprite = split_animated_gif('source/Pokemon/'+id+'/Status.gif')
        self.AttackSprite = split_animated_gif('source/Pokemon/'+id+'/Attack.gif')
        self.SpecialAnimationSprite = split_animated_gif('source/Pokemon/'+id+'/SpecialAnimation.gif')

        self.StatusSprite['list'] = ScaleTimes(self.StatusSprite['list'], 2)
        self.AttackSprite['list'] = ScaleTimes(self.AttackSprite['list'], 2)

        self.width = self.FrontSprite['list'][0].get_width()
        self.height = self.FrontSprite['list'][0].get_height()
        self.id = id
        self.loadElement()
        self.loadBase()
        self.loadStats()
        self.loadSkills()
    
    def loadElement(self):
        sql = "SELECT e.Name FROM Pokemon AS p INNER JOIN PokeType AS pt ON p.pokeId=pt.pokeId and p.pokeId=\'"+self.id+"\' INNER JOIN Element AS e ON e.eleId=pt.eleId"
        cursor.execute(sql)
        self.Element = []
        for i in cursor: 
            self.Element.append(tuple(i)[0])

    def loadBase(self):
        sql = "SELECT es.Based FROM ElementSource AS es INNER JOIN Element as e ON e.eleId=es.eleId and e.Name=\'"+str(self.Element[0])+"\'"
        cursor.execute(sql)
        for cur in cursor:
            path = tuple(cur)[0]
            self.Based = pygame.image.load(path).convert_alpha()
            break
        
    def loadStats(self):
        sql = "SELECT p.Name, pi.* FROM Pokemon AS p INNER JOIN PokeInfor AS pi ON p.pokeId=pi.pokeId and p.pokeId=\'"+self.id+"\'"
        cursor.execute(sql)
        for cur in cursor:
            self.name, id, self.hp, self.attack, self.spAttack, self.defense, self.spDefense, self.speed = tuple(cur)
            name, id, self.base_hp, self.base_attack, self.base_spAttack, self.base_defense, self.base_spDefense, self.base_speed = tuple(cur)
            self.maxHp = self.hp
            break
    
    def loadSkills(self):
        sql = "SELECT ps.* FROM PokeSkill AS ps INNER JOIN Pokemon AS p ON ps.pokeId=p.pokeId and p.pokeId=\'"+self.id+"\'"
        cursor.execute(sql)
        self.skillList = []
        for cur in cursor:
            self.skillList.append(tuple(cur)[1])
        self.skillList = [Skill(id) for id in self.skillList]

    def getBaseStatByNameList(self, nameList):
        if len(nameList)>1: return self.spDefense
        dic = {'HP': self.base_hp, 'Physical': self.base_attack,'Special': self.base_spAttack,'Defense': self.base_defense, 'Special Defense': self.base_spDefense, 'Speed': self.base_speed}
        for key in dic.keys():
            if key==nameList[0]: return dic[key]

    def getStatByNameList(self, nameList):
        if len(nameList)>1: return self.spDefense
        dic = {'Name': self.name, 'Id': self.id, 'HP': self.hp, 'Physical': self.attack,'Special': self.spAttack,'Defense': self.defense, 'Special Defense': self.spDefense, 'Speed': self.speed}
        for key in dic.keys():
            if key==nameList[0]: return dic[key]

    def setStatByNameList(self, nameList, value):
        if len(nameList)>1: self.spDefense=value
        elif nameList[0]=='Defense': self.defense=value
        elif nameList[0]=='Special': self.spAttack=value
        elif nameList[0]=='Physical': self.attack=value

    def ResetSkill(self):
        for skill in self.skillList:
            skill.setCountDownSkill_getTurn()

<h2>Lifebar class
</h2>

In [98]:
class LifeBar:
	def __init__(self, pokeId, maxHP, pos, speed=2, height=15, width=130):
		self.pokeId = pokeId
		self.current_health = 0
		self.target_health = maxHP
		self.max_health = maxHP
		self.health_bar_length = width
		self.health_bar_height = height
		self.health_bar_border = 2
		self.posX, self.posY = pos
		self.health_ratio = self.max_health / self.health_bar_length
		self.health_change_speed = speed

	def get_damage(self,amount):
		if self.target_health > 0:
			self.target_health -= amount
		if self.target_health < 0:
			self.target_health = 0

	def get_health(self,amount):
		if self.target_health < self.max_health:
			self.target_health += amount
		if self.target_health > self.max_health:
			self.target_health = self.max_health

	def update(self, screen):
		self.advanced_health(screen)

	def advanced_health(self, screen):
		transition_width = 0
		transition_color = (255,0,0)

		if self.current_health < self.target_health:
			self.current_health += self.health_change_speed
			transition_width = int((self.target_health - self.current_health) / self.health_ratio)
			transition_color = (0,255,0)

		if self.current_health > self.target_health:
			self.current_health -= self.health_change_speed 
			transition_width = int((self.target_health - self.current_health) / self.health_ratio)
			transition_color = (255,255,0)

		health_bar_width = int(self.current_health / self.health_ratio)
		health_bar = pygame.Rect(self.posX,self.posY,
								health_bar_width,
								self.health_bar_height)

		transition_bar = pygame.Rect(health_bar.right,self.posY,
								transition_width,
								self.health_bar_height)
		
		pygame.draw.rect(screen,(255,0,0),health_bar)
		pygame.draw.rect(screen,transition_color,transition_bar)	
		
		pygame.draw.rect(screen,(255,255,255),
							(self.posX,self.posY,
								self.health_bar_length,
								self.health_bar_height),
							self.health_bar_border)	

<h2>Team class</h2>

In [99]:
class Team:
    def __init__(self, teamList, pos, gap=100):
        self.team = teamList
        self.posX, self.posY = pos
        self.gap = 150
        self.generateHPbar()
        self.battleEffect = [[] for _ in range(len(self.team))]
        # this variable would store the position of each pokemon
        self.teamPos = [{'id': pokemon.id, 'pos': (self.getNewPosx(i), self.posY)} for pokemon, i in zip(teamList, range(len(teamList)))]
        
    def getNewPosx(self, iter):
        return self.posX+self.gap*iter

    def getConstPos(self, pokemon):
        for posPack in self.teamPos:
            if (posPack['id']==pokemon.id):
                return posPack['pos']

    def generateHPbar(self):
        y = self.posY-130 if self.posY<=SCREEN_HEIGHT/2 else self.posY+30
        self.HPbar = [LifeBar(pokemon.id, pokemon.hp, (self.getNewPosx(i)-70, y)) for pokemon, i in zip(self.team, range(len(self.team)))]

<h2>Blit cluster of things</h2>

In [100]:
def BlitPokemonWithBased(screen, pokemon, pos, direct='Front'):
    BlitBased(screen, pokemon, (pos[0], pos[1]+30))
    if direct=='Front':
        pokemon.FrontSprite = BlitGif(screen, pokemon.FrontSprite, (pos[0], pos[1]))
    elif direct=='Back':
        pokemon.BackSprite = BlitGif(screen, pokemon.BackSprite, (pos[0], pos[1]))


def BLitSkill(screen, pokemon, selection=0):
    BlitBox(screen, (0, 450), (400, 150))
    posX, posY = 40, 425
    for skill, i in zip(pokemon.skillList, range(len(pokemon.skillList))):
        if (i==selection):
            color = BLUE if pokemon.skillList[selection].isAvailable else RED
            BlitText(screen, skill.name, (posX, posY), color=color)
            text = skill.explain if color==BLUE else f'{skill.explain} - (not vailable, in {skill.count} turn(s))'
            Blit_mutiline_text(screen, text, (10, 460), (400, 150))
        else:
            BlitText(screen, skill.name, (posX, posY))
        posX+=120


def BlitTeam(screen, A, B):
    for pokemon in A.team:
        pos = A.getConstPos(pokemon)
        BlitPokemonWithBased(screen, pokemon, pos)
    for pokemon in B.team:
        pos = B.getConstPos(pokemon)
        BlitPokemonWithBased(screen, pokemon, pos, direct='Back')


def getSpecialAnimation(pokemonId, A, B, isAct=False):
    special = []
    for pokemonA, pokemonB in zip(A.team, B.team):
        if (pokemonA.id == pokemonId):
            special = pokemonA.SpecialAnimationSprite
            break
        if (pokemonB.id == pokemonId): 
            special = pokemonB.SpecialAnimationSprite
            break 
    special['frame']=0
    return {
        'animation': special,
        'isAnimating': isAct
    }


def BlitSpecialAnimation(screen, user, A, B, AnimatePackage):
    animation = BlitGif(screen, AnimatePackage['animation'], 
                (SCREEN_LENGTH/2, SCREEN_HEIGHT/2-70), 
                speed=0.2, 
                scalePoint='center',
                isOneAction=True)

    isAct = True if (animation['frame']<len(animation['list'])-1) else False
    return {'animation': animation, 'isAnimating': isAct}

def BlitArrowTurn(screen, arrowObject, currentPlayerPackage, A, B):
    if (currentPlayerPackage=='none'): return
    gap = 150 if (currentPlayerPackage['team']=='A') else 90
    player = UnpackPackage(currentPlayerPackage, A, B)
    pos = player['team'].getConstPos(player['pokemon'])
    BlitGif(screen, arrowObject, (pos[0], pos[1]-gap))

def BlitEffectPerTeam(screen, Team, gapX=0, gapY=0):
    for i in range(len(Team.battleEffect)):
        posX, posY = Team.getConstPos(Team.team[i])
        for effect in Team.battleEffect[i]:
            if (effect.sprite=='none'): continue
            screen.blit(effect.sprite, effect.sprite.get_rect(center=(posX+gapX, posY+gapY)))
            posX+=25

def BlitEffects(screen, A, B):
    BlitEffectPerTeam(screen, A, -60, -145)
    BlitEffectPerTeam(screen, B, -60, 60)


<h2>Effect SetUp</h2>

In [101]:
def Adjust_Stats_By_Effect(pokemon, effect):
    if (effect.name=='Slow'):
        pokemon.speed -= GetPercentage(pokemon.speed, 30)
        return
    if ('Increase' not in effect.name) and ('Decrease' not in effect.name): return
    sign = 1 if 'Increase' in effect.name.split(' ') else -1
    stat = effect.name.split(' ')
    stat.pop(0)
    originalStat = pokemon.getStatByNameList(stat)
    pokemon.setStatByNameList(stat, originalStat+sign*GetPercentage(originalStat, 30))


def Adjust_Stats_By_Remove_Effect(pokemon, effect):
    if (effect.name=='Slow'):
        pokemon.speed = pokemon.base_speed
        return
    stat = effect.name.split(' ')
    stat.pop(0)
    # print('original stat: ', pokemon.getStatByNameList(stat))
    # print('base stat: ', pokemon.getBaseStatByNameList(stat))
    baseStat = pokemon.getBaseStatByNameList(stat)
    pokemon.setStatByNameList(stat, baseStat)
    # print('remove stats')


def add_effects(user, userTeam, target, targetTeam, effect, A, B):
    TEAM = targetTeam if (effect.rangeEffect=='Single Target' or effect.rangeEffect=='AOE Targets') else userTeam
    RANGE = 'AOE' if (effect.rangeEffect=='AOE Targets' or effect.rangeEffect=='AOE Self') else 'Single'
    TARGET = user if (effect.rangeEffect=='Self') else target
    
    for i in range(len(TEAM.team)):
        #AOE treatement
        if (RANGE=='AOE' and 
            isAlive(TEAM.team[i]) and
            not checkEffectExist(effect, TEAM.battleEffect[i])):
                if (not GetChance(effect.rate)): continue
                TEAM.battleEffect[i].append(effect)
                Adjust_Stats_By_Effect(TEAM.team[i], effect)
                color = PINK if effect.typeEffect=='Harmful' else CRYAN 
                MESS.Add_messbox_with_unpackage(effect.name, {'pokemon': TEAM.team[i], 'team': TEAM}, A, B, color=color)
                continue
        #Single target treatement
        elif (TEAM.team[i].id == TARGET.id and 
            isAlive(TEAM.team[i]) and
            not checkEffectExist(effect, TEAM.battleEffect[i])):
                if (not GetChance(effect.rate)): return
                TEAM.battleEffect[i].append(effect)
                Adjust_Stats_By_Effect(TARGET, effect)
                color = PINK if effect.typeEffect=='Harmful' else CRYAN 
                MESS.Add_messbox_with_unpackage(effect.name, {'pokemon': TARGET, 'team': TEAM}, A, B, color=color)
                return

def remove_effects(user, userTeam, target, targetTeam, effect, A, B):
    TEAM = targetTeam if (effect.rangeEffect=='Single Target' or effect.rangeEffect=='AOE Targets') else userTeam
    RANGE = 'AOE' if (effect.rangeEffect=='AOE Targets' or effect.rangeEffect=='AOE Self') else 'Single'
    TARGET = user if (effect.rangeEffect=='Self') else target
    TYPE = 'Benefical' if (effect.name=='Strip Benefical') else 'Harmful'

    for i in range(len(TEAM.team)):
        #AOE treatement
        if (RANGE=='AOE' and 
            isAlive(TEAM.team[i]) and
            not checkEffectExist(effect, TEAM.battleEffect[i])):
                for eff in TEAM.battleEffect[i]:
                    if (eff.typeEffect==TYPE):
                        TEAM.battleEffect[i].remove(eff)
                        Adjust_Stats_By_Remove_Effect(TEAM.team[i], eff)
                continue
        #Single target treatement
        elif (TEAM.team[i].id == TARGET.id and 
            isAlive(TEAM.team[i]) and
            not checkEffectExist(effect, TEAM.battleEffect[i])):
                for eff in TEAM.battleEffect[i]:
                    if (eff.typeEffect==TYPE):
                        TEAM.battleEffect[i].remove(eff)
                        Adjust_Stats_By_Remove_Effect(TEAM.team[i], eff)
                return


def is_suffering_bad_effect(pokemon, Team, effectSuffering='none', remove=False):
    for i in range(len(Team.battleEffect)):
        if (Team.team[i].id==pokemon.id):
            for effect in Team.battleEffect[i]:
                if (effect.name==effectSuffering):
                    if remove: Team.battleEffect[i].remove(effect)
                    return True
    return False

def getDOT(pokemon, Team, PercentHP=5):
    print(f'{pokemon.name} is hurt by Burn/Poison')
    for i in range(len(Team.team)):
        if Team.team[i].id == pokemon.id:
            value = GetPercentage(Team.team[i].maxHp, PercentHP)
            Team.HPbar[i].get_damage(value)
            Team.team[i].hp -= value

<h2>Game Logic Calculation</h2>

In [102]:
def Specific_calculation_damage(SkillPercent, stab, attack, defense):
    critical = random.randint(1,2)
    power = GetPercentage(attack, SkillPercent)
    return ((((((2*critical)/5)+2)*power*(attack/defense))/50)+2)*stab


def Calculate_skilldamage(user, skill, target, isIgnoreDef=False, isIgnoreRed=False):
    stab = CheckingStab(user, target)
    if skill.kind == 'Physical':
        attack, defense = user.attack, target.defense
        if isIgnoreDef: defense = GetPercentage(defense, 10)
        if isIgnoreDef: attack = user.base_attack
    else:
        attack, defense = user.spAttack, target.spDefense
        if isIgnoreDef: spDefense = GetPercentage(defense, 10)
        if isIgnoreDef: spAttack = user.base_spAttack
    skill_damage = skill.damage
    return Specific_calculation_damage(skill_damage, stab, attack, defense)


def Healing_Amount(skill, user, userTeam, amount, A, B, rangeSkill='Self'):
    for i in range(len(userTeam.team)):
        amount = GetPercentage(userTeam.team[i].maxHp, amount)
        if (rangeSkill=='AOE Self' and isAlive(userTeam.team[i])):
            userTeam.HPbar[i].get_health(amount)
            userTeam.team[i].hp+=amount
            MESS.Add_messbox_with_unpackage('Healing', {'pokemon': userTeam.team[i], 'team': userTeam}, A, B, color=GREEN)
            ANIMATION.Add_animatebox_with_unpackage(skill, {'pokemon': userTeam.team[i], 'team': userTeam}, A, B)
            continue
        elif rangeSkill=='Self' and isAlive(userTeam.team[i]) and userTeam.team[i].id==user.id:
            userTeam.HPbar[i].get_health(amount)
            userTeam.team[i].hp+=amount
            MESS.Add_messbox_with_unpackage('Healing', {'pokemon': userTeam.team[i], 'team': userTeam}, A, B, color=GREEN)
            ANIMATION.Add_animatebox_with_unpackage(skill, {'pokemon': userTeam.team[i], 'team': userTeam}, A, B)
            return

def Recieve_damage(userUnpackage, targetUnpackage, skill, A, B, isIgnoreDef=False, isIgnoreRed=False):
    user = userUnpackage['pokemon']
    target, targetTeam = targetUnpackage['pokemon'], targetUnpackage['team']
    for i in range(len(targetTeam.team)):
        damage = Calculate_skilldamage(user, skill, targetTeam.team[i], isIgnoreDef=isIgnoreDef, isIgnoreRed=isIgnoreRed)
        if (skill.range=='AOE Targets' and isAlive(targetTeam.team[i])):
            targetTeam.HPbar[i].get_damage(damage)
            targetTeam.team[i].hp-=damage
            ANIMATION.Add_animatebox_with_unpackage(skill, {'pokemon': targetTeam.team[i], 'team': targetTeam}, A, B)
            if (not isAlive(targetTeam.team[i])):
                print(f"{targetTeam.team[i].name} is defeated!")
            continue
        elif (skill.range=='Single Target' and isAlive(targetTeam.team[i]) and targetTeam.team[i].id==target.id):
            targetTeam.HPbar[i].get_damage(damage)
            targetTeam.team[i].hp-=damage
            ANIMATION.Add_animatebox_with_unpackage(skill, {'pokemon': target, 'team': targetTeam}, A, B)
            if (not isAlive(targetTeam.team[i])):
                print(f"{targetTeam.team[i].name} is defeated!")
            return


def SetEffects(skill, userUnpackage, targetUnpackage, effect, A, B, amount=0):
    if effect.name=='Heal':
        Healing_Amount(skill, userUnpackage['pokemon'], userUnpackage['team'], amount, A, B, effect.rangeEffect)
    elif effect.name=='Ignore Defense' or effect.name=='Ignore Reduction':
        MESS.Add_messbox_with_unpackage(effect.name, userUnpackage, A, B, color=CRYAN)
        print(f"This damage is {effect.name}")
    elif effect.name=='Strip Benefical' or effect.name=='Strip Harmful':
        remove_effects(userUnpackage['pokemon'], userUnpackage['team'], targetUnpackage['pokemon'], targetUnpackage['team'], effect, A, B)
    else:
        add_effects(userUnpackage['pokemon'], userUnpackage['team'], targetUnpackage['pokemon'], targetUnpackage['team'], effect, A, B)
    

def Skill_Calculation(userPackage, skillSelected, targetPackage, A, B):
    userUnpackage = UnpackPackage(userPackage, A, B)
    targetUnpackage = UnpackPackage(targetPackage, A, B)
    skill = userUnpackage['pokemon'].skillList[skillSelected]
    print(userUnpackage['pokemon'].name, "using", skill.name)
    skill.setCountDownSkill_useSkill()
    #check if self buff, set animation for this move
    if skill.range == 'Self': ANIMATION.Add_animatebox_with_package(skill, userPackage, A, B)
    amountEffect = userUnpackage['pokemon'].skillList[skillSelected].damage
    #set effect of skill first
    isIgnoreDef, isIgnoreRed = False, False
    for effect in skill.EffectList:
        SetEffects(skill, userUnpackage, targetUnpackage, effect, A, B, amount=amountEffect) 
        isIgnoreDef = True if effect.name=="Ignore Defense" else False
        isIgnoreRed = True if effect.name=="Ignore Reduction" else False
    #then calculate the damage
    Recieve_damage(userUnpackage, targetUnpackage, skill, A, B, isIgnoreDef=isIgnoreDef, isIgnoreRed=isIgnoreRed)


def CheckAliveMembers(Team, MoveOrder):
    index = 0
    while index < len(Team.team):
        if Team.team[index].hp<=0:
            print(f"{Team.team[index].name} is defeated!")
            for order in MoveOrder:
                if order['id']==Team.team[index].id:
                    MoveOrder.remove(order)
            Team.team.pop(index)
            Team.HPbar.pop(index)
            Team.battleEffect.pop(index)
            index-=1
        else: index+=1
    return Team, MoveOrder

<h2>Game Events</h2>

In [103]:
def SkillSelection(event, selection):
    if event.type == pygame.KEYDOWN:
        if event.key == pygame.K_RIGHT:
            return 0 if selection+1>=3 else selection+1
        elif event.key == pygame.K_LEFT:
            return 2 if selection-1<0 else selection-1
    return selection
    

def GetTurn(order, A, B):
    try:
        order = sorted(order, key=lambda d: d['speed'])
        currentPlayer = order.pop()
        #check pokemon is stun or not
        pokemon = getPokemonById(currentPlayer['id'], A, B)
        team = getTeamById(currentPlayer['team'], A, B)
        while(is_suffering_bad_effect(pokemon, team, 'Stun', remove=True) 
            or isAlive(pokemon)==False): 
                print(f'{pokemon.name} is stuning')
                currentPlayer = order.pop()
                pokemon = getPokemonById(currentPlayer['id'], A, B)
                team = getTeamById(currentPlayer['team'], A, B)
        print(f'\nIts {pokemon.name} turn!')
        if (is_suffering_bad_effect(pokemon, team, 'Burn', remove=False)):
            getDOT(pokemon, team)
        pokemon.ResetSkill()
        return order, currentPlayer
    except:
        if len(order)>0:
            target = order.pop()
            print(f'Its {pokemon.name} turn!')
            if (is_suffering_bad_effect(pokemon, team, 'Burn', remove=False)):
                getDOT(pokemon, team)
                pokemon.ResetSkill()
            return order, target
        return order, 'none'


def SetupTurn(screen, target, A, B, skillSelected):
    if target=='none': return
    for pokemon in A.team:
        if pokemon.id==target['id']:
            BLitSkill(screen, pokemon, selection=skillSelected)
            return
    for pokemon in B.team:
        if pokemon.id==target['id']:
            BLitSkill(screen, pokemon, selection=skillSelected)
            return


def SetupNewTurn(screen, A, B, MoveOrder, isNewTurn):
    if isNewTurn==False: return MoveOrder
    for pokemon, i in zip(A.team, range(len(A.team))):
        MoveOrder.append({'id': pokemon.id, 'speed': pokemon.speed, 'team': 'A'})
    for pokemon, i in zip(B.team, range(len(B.team))):
        MoveOrder.append({'id': pokemon.id, 'speed': pokemon.speed, 'team': 'B'})
    return MoveOrder


def SetupHoverWithPokemon(screen, A, B, mouse):
    for pokemon in A.team:
        pos = A.getConstPos(pokemon)
        if (mouse[0]>(pos[0]-pokemon.width/2) and 
            mouse[0]<(pos[0]+pokemon.width/2) and
            mouse[1]>(pos[1]-pokemon.height) and mouse[1]<pos[1]):
                pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_HAND)
                pygame.draw.circle(screen, (240,240,240), (pos[0],pos[1]-20), 70) 
                return {'id': pokemon.id, 'team':'A'}
    for pokemon in B.team:
        pos = B.getConstPos(pokemon)
        if (mouse[0]>(pos[0]-pokemon.width/2) and 
            mouse[0]<(pos[0]+pokemon.width/2) and
            mouse[1]>(pos[1]-pokemon.height) and mouse[1]<pos[1]):
                pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_HAND)
                pygame.draw.circle(screen, (240,240,240), (pos[0],pos[1]-20), 70) 
                return {'id': pokemon.id, 'team':'B'}
    pygame.mouse.set_cursor(pygame.SYSTEM_CURSOR_ARROW)
    return {'id': '', 'team': ''}


def ClickTargetEvent(userPackage, hoverPackage, skillSelected, A, B):
    if hoverPackage['id'] == '' and hoverPackage['team'] == '': return False
    if not UnpackPackage(userPackage, A, B)['pokemon'].skillList[skillSelected].isAvailable: return False
    Skill_Calculation(userPackage, skillSelected, hoverPackage, A, B)
    return True

<h2>Main Game</h2>

In [104]:
pygame.init()
screen = pygame.display.set_mode((SCREEN_LENGTH, SCREEN_HEIGHT))
running = True
clock = pygame.time.Clock()
MESS.SetScreen(screen)
ANIMATION.SetScreen(screen)

# try:
A = Team([Pokemon('006'), Pokemon('009'), Pokemon('241')], (300, 175))
B = Team([Pokemon('003'), Pokemon('212'), Pokemon('807')], (200, 355))
Background = pygame.image.load('source/Background/CityBattleScreen.png').convert()
turnArrow = split_animated_gif('source/vfx/Arrow.gif')

skillSelected = 0
MoveOrder, currentPlayer = [], 'none'
AnimatePackage = getSpecialAnimation(A.team[0].id, A, B)

while running:
    screen.fill((0,0,0))
    screen.blit(Background, Background.get_rect(center=(SCREEN_LENGTH/2, SCREEN_HEIGHT/2-75)))
    isNewTurn = True if (len(MoveOrder)==0 and currentPlayer=='none') else False
    mouse = pygame.mouse.get_pos()
    
    # A, MoveOrder = CheckAliveMembers(A, MoveOrder)
    # B, MoveOrder = CheckAliveMembers(B, MoveOrder)
    hoverPackage = SetupHoverWithPokemon(screen, A, B, mouse)
    BlitTeam(screen, A, B)
    MoveOrder = SetupNewTurn(screen, A, B, MoveOrder, isNewTurn)
    if isNewTurn: MoveOrder, currentPlayer = GetTurn(MoveOrder, A, B)
    BlitArrowTurn(screen, turnArrow, currentPlayer, A, B)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running=False
        skillSelected = SkillSelection(event, skillSelected)
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                MoveOrder, currentPlayer = GetTurn(MoveOrder, A, B)
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                if not ClickTargetEvent(currentPlayer, hoverPackage, skillSelected, A, B): break
                if (skillSelected==2):
                    AnimatePackage = getSpecialAnimation(currentPlayer['id'], A, B, isAct=True)
                MoveOrder, currentPlayer = GetTurn(MoveOrder, A, B)

    BlitEffects(screen, A, B)
    
    if (AnimatePackage['isAnimating']==False):
        ANIMATION.BlitAnimation()
        MESS.BlitMess()
        SetupTurn(screen, currentPlayer, A, B, skillSelected)
        [hpbar.update(screen) for hpbar in A.HPbar]
        [hpbar.update(screen) for hpbar in B.HPbar]
    else:
        AnimatePackage = BlitSpecialAnimation(screen, currentPlayer, A, B, AnimatePackage)

    clock.tick(60)
    pygame.display.flip()
pygame.quit()

# except Exception as e:
#     print("Oops!", sys.exc_info()[0], "occurred.")
#     print("The Error:", e)
#     pygame.quit()


Its Zeraora turn!
